# **Dataset Preprocessing**

## **Import Libraries**

In [45]:
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

pd.set_option('display.max_colwidth', None)

In [46]:
# jika belum pernah:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Load Dataset**

In [47]:
input_file = "cqa_datasets.jsonl"
df = pd.read_json(input_file, lines=True)
print(f"Total QA pairs: {len(df)}")
df.head(3)

Total QA pairs: 1696


,context,question_answerable,answer_answerable,category_answerable,question_unanswerable,answer_unanswerable,category_unanswerable,file_url,regulation_number,title,filename,n_pairs_requested
0,"Pasal 1 Dalam Peraturan Otoritas Jasa Keuangan ini yang dimaksud dengan: 1. Emiten adalah Pihak yang melakukan Penawaran Umum. 2. Perusahaan Publik adalah Perseroan yang sahamnya telah dimiliki paling sedikit oleh 300 (tiga ratus) pemegang saham dan memiliki modal disetor paling sedikit Rp3.000.000.000,00 (tiga miliar rupiah) atau suatu jumlah pemegang saham dan modal disetor yang ditetapkan dengan Peraturan Pemerintah. 3. Laporan adalah laporan, keterbukaan informasi, atau dokumen yang wajib disampaikan oleh Emiten atau Perusahaan Publik kepada Otoritas Jasa Keuangan sebagaimana dimaksud dalam ketentuan peraturan perundang-undangan di sektor pasar modal. 4. Sistem Pelaporan Elektronik yang selanjutnya disingkat SPE adalah sistem informasi yang digunakan sebagai sarana penyampaian Laporan secara elektronik oleh Emiten atau Perusahaan Publik kepada Otoritas Jasa Keuangan.",Apa pengertian Sistem Pelaporan Elektronik (SPE) menurut peraturan ini?,Sistem Pelaporan Elektronik adalah sistem informasi yang digunakan sebagai sarana penyampaian Laporan secara elektronik oleh Emiten atau Perusahaan Publik kepada Otoritas Jasa Keuangan.,Pengertian,Apa pengertian SRO menurut peraturan ini?,Saya tidak tahu terkait pengertian SRO menurut peraturan ini.,Pengertian,https://www.ojk.go.id/id/regulasi/Documents/Pages/Penyampaian-Laporan-melalui-Sistem-Pelaporan-Elektronik-Emiten-atau-Perusahaan-Publik/POJK%207-2018.pdf,7/POJK.04/2018,Penyampaian Laporan melalui Sistem Pelaporan Elektronik Emiten atau Perusahaan Publik,ojk-peraturan_ojk-7_pojk_04_2018-25042018-penyampaian_laporan_melalui_sistem_pelaporan_elektronik_emiten_atau_perusahaan_publik.pdf,8
1,"Pasal 2 (1) Emiten atau Perusahaan Publik wajib menyampaikan Laporan kepada Otoritas Jasa Keuangan melalui SPE. (2) SPE sebagaimana dimaksud pada ayat (1) dapat diakses melalui situs web dengan alamat https://spe.ojk.go.id atau alamat lain yang ditetapkan Otoritas Jasa Keuangan. (3) Laporan sebagaimana dimaksud pada ayat (1) meliputi Laporan yang diwajibkan berdasarkan ketentuan peraturan perundang-undangan di sektor pasar modal sebagai berikut: a. peraturan mengenai rencana dan penyelenggaraan rapat umum pemegang saham; b. peraturan mengenai laporan realisasi penggunaan dana hasil penawaran umum; c. peraturan mengenai kewajiban penyampaian laporan keuangan berkala Emiten atau Perusahaan Publik; d. peraturan mengenai penyampaian laporan tahunan Emiten atau Perusahaan Publik; e. peraturan mengenai penggunaan jasa akuntan publik dan kantor akuntan publik dalam kegiatan jasa keuangan; f. peraturan mengenai penerapan keuangan berkelanjutan bagi lembaga jasa keuangan, Emiten, dan Perusahaan Publik; g. peraturan mengenai keterbukaan atas informasi atau fakta material oleh Emiten atau Perusahaan Publik; h. peraturan mengenai transaksi afiliasi dan benturan kepentingan transaksi tertentu; i. peraturan mengenai transaksi material dan perubahan kegiatan usaha utama;",Apa kewajiban yang ditetapkan bagi Emiten atau Perusahaan Publik dalam Pasal 2 ayat (1)?,Kewajiban yang ditetapkan adalah menyampaikan Laporan kepada Otoritas Jasa Keuangan melalui SPE.,Kewajiban yang Ditetapkan,Apa kewajiban yang ditetapkan bagi SRO dalam Pasal 2 ayat (1)?,Saya tidak tahu terkait kewajiban yang ditetapkan bagi SRO dalam Pasal 2 ayat (1).,Kewajiban yang Ditetapkan,https://www.ojk.go.id/id/regulasi/Documents/Pages/Penyampaian-Laporan-melalui-Sistem-Pelaporan-Elektronik-Emiten-atau-Perusahaan-Publik/POJK%207-2018.pdf,7/POJK.04/2018,Penyampaian Laporan melalui Sistem Pelaporan Elektronik Emiten atau Perusahaan Publik,ojk-peraturan_ojk-7_pojk_04_2018-25042018-penyampaian_laporan_melalui_sistem_pelaporan_elektronik_emiten_atau_perusahaan_publik.pdf,8
2,j. peraturan mengenai kepemilikan atau setiap perubahan kepemilikan saham perusahaan terbuka

## **Filter Data with High Similarity Context**

In [48]:
## **Filter Data with High Similarity Context & Questions**

# 3) Prepare TF-IDF & Stop-Words
stop_id = set(stopwords.words("indonesian"))

# thresholds
threshold_ctx = 0.90
threshold_qa  = 0.95  # for question_answerable
threshold_qu  = 0.95  # for question_unanswerable

# TF-IDF untuk context
vect_ctx = TfidfVectorizer(
    max_features=5000,
    stop_words=list(stop_id),
    token_pattern=r"(?u)\b\w+\b"
)
X_ctx = vect_ctx.fit_transform(df['context'])

# TF-IDF untuk question_answerable
vect_qa = TfidfVectorizer(
    max_features=2000,
    stop_words=list(stop_id),
    token_pattern=r"(?u)\b\w+\b"
)
X_qa = vect_qa.fit_transform(df['question_answerable'])

# TF-IDF untuk question_unanswerable
vect_qu = TfidfVectorizer(
    max_features=2000,
    stop_words=list(stop_id),
    token_pattern=r"(?u)\b\w+\b"
)
X_qu = vect_qu.fit_transform(df['question_unanswerable'])

# 4) Find High-Similarity Pairs
pairs_ctx = set()
pairs_qa  = set()
pairs_qu  = set()

def find_pairs(X, thr, out_set, desc):
    n = X.shape[0]
    for start in tqdm(range(0, n, 500), desc=desc):
        end   = min(start+500, n)
        block = X[start:end]
        sims  = cosine_similarity(block, X)
        for i, row in enumerate(sims):
            idx1 = start + i
            for idx2, sim in enumerate(row):
                if idx2 > idx1 and sim > thr:
                    out_set.add((idx1, idx2))

find_pairs(X_ctx, threshold_ctx, pairs_ctx, desc="Scanning context")
find_pairs(X_qa,  threshold_qa,  pairs_qa,  desc="Scanning question_answerable")
find_pairs(X_qu,  threshold_qu,  pairs_qu,  desc="Scanning question_unanswerable")

# intersect: hanya yang high similarity di ketiga bidang
dup_pairs = pairs_ctx & pairs_qa & pairs_qu
print(f"\nFound {len(dup_pairs)} pairs with high similarity in context, answerable & unanswerable questions\n")


c:\Users\acer\miniconda3\envs\ta-datasets-creation\lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
Scanning question_unanswerable: 100%|██████████| 4/4 [00:00<00:00, 14.55it/s]


Found 2 pairs with high similarity in context, answerable & unanswerable questions



In [49]:
## 5) (Optional) Inspect Top 10 Pairs
pairs_list = []
for a, b in list(dup_pairs)[:10]:
    # similarity di ketiga bidang
    sim_ctx = cosine_similarity(X_ctx[a], X_ctx[b])[0,0]
    sim_qa  = cosine_similarity(X_qa[a],  X_qa[b])[0,0]
    sim_qu  = cosine_similarity(X_qu[a],  X_qu[b])[0,0]

    pairs_list.append({
        "idx1": a,
        "idx2": b,
        "sim_ctx": sim_ctx,
        "sim_question_answerable": sim_qa,
        "sim_question_unanswerable": sim_qu,
        "context_1": df.loc[a, "context"],
        "context_2": df.loc[b, "context"],
        "question_answerable_1": df.loc[a, "question_answerable"],
        "question_answerable_2": df.loc[b, "question_answerable"],
        "question_unanswerable_1": df.loc[a, "question_unanswerable"],
        "question_unanswerable_2": df.loc[b, "question_unanswerable"],
        "file_url_1": df.loc[a, "file_url"],
        "file_url_2": df.loc[b, "file_url"],
    })

# Tampilkan sebagai DataFrame
pd.DataFrame(pairs_list)


,idx1,idx2,sim_ctx,sim_question_answerable,sim_question_unanswerable,context_1,context_2,question_answerable_1,question_answerable_2,question_unanswerable_1,question_unanswerable_2,file_url_1,file_url_2
0,1021,1028,0.965131,1.0,1.0,"Pasal II \nPeraturan Otoritas Jasa Keuangan ini mulai berlaku pada tanggal diundangkan.\n\n-4- \n\n\n\nSalinan ini sesuai dengan aslinya \nDirektur Hukum 1 \nDepartemen Hukum \n\nttd \n\nYuliana \nAgar \nsetiap \norang \nmengetahuinya, \nmemerintahkan \npengundangan Peraturan Otoritas Jasa Keuangan ini dengan \npenempatannya dalam Lembaran Negara Republik Indonesia \n\n\nDitetapkan di Jakarta \npada tanggal 5 Desember 2018 \n\nKETUA DEWAN KOMISIONER \nOTORITAS JASA KEUANGAN \nREPUBLIK INDONESIA, \n\nttd \n\nWIMBOH SANTOSO \n\nDiundangkan di Jakarta \npada tanggal 10 Desember 2018 \n\nMENTERI HUKUM DAN HAK ASASI MANUSIA \nREPUBLIK INDONESIA \n\nttd \n\nYASONNA H. LAOLY \n\n\nLEMBARAN NEGARA REPUBLIK INDONESIA TAHUN 2018 NOMOR 242","Pasal II Peraturan Otoritas Jasa Keuangan ini mulai berlaku pada tanggal diundangkan. -9- Salinan ini sesuai dengan aslinya Direktur Hukum 1 Departemen Hukum ttd Yuliana Agar setiap orang mengetahuinya, memerintahkan pengundangan Peraturan Otoritas Jasa Keuangan ini dengan penempatannya dalam Lembaran Negara Republik Indonesia. Ditetapkan di Jakarta pada tanggal 5 Desember 2018 KETUA DEWAN KOMISIONER OTORITAS JASA KEUANGAN REPUBLIK INDONESIA, ttd WIMBOH SANTOSO Diundangkan di Jakarta pada tanggal 10 Desember 2018 MENTERI HUKUM DAN HAK ASASI MANUSIA REPUBLIK INDONESIA ttd YASONNA H. LAOLY LEMBARAN NEGARA REPUBLIK INDONESIA TAHUN 2018 NOMOR 240",Kapan peraturan ini mulai berlaku?,Kapan masa berlaku peraturan ini dimulai?,Sampai kapan masa berlaku peraturan ini untuk sektor perbankan?,Sampai kapan masa berlaku peraturan ini di sektor perbankan?,https://www.ojk.go.id/id/regulasi/Documents/Pages/Perubahan-Kedua-POJK-tentang-Tata-Cara-Penagihan-Sanksi-Administratif-Berupa-Denda-di-Sektor-Jasa-Keuangan/pojk%2026-2018.pdf,https://www.ojk.go.id/id/regulasi/Documents/Pages/Perubahan-POJK-Nomor-3-POJK.02-2014-tentang-Tata-Cara-Pelaksanaan-Pungutan-oleh-OJK/pojk%2022-2018.pdf
1,1602,1615,1.000000,1.0,1.0,"Pasal 20 Peraturan Otoritas Jasa Keuangan ini mulai berlaku pada tanggal diundangkan. Agar setiap orang mengetahuinya, memerintahkan pengundangan Peraturan Otoritas Jasa Keuangan ini dengan penempatannya dalam Lembaran Negara Republik Indonesia.","Pasal 20 Peraturan Otoritas Jasa Keuangan ini mulai berlaku pada tanggal diundangkan. Agar setiap orang mengetahuinya, memerintahkan pengundangan Peraturan Otoritas Jasa Keuangan ini dengan penempatannya dalam Lembaran Negara Republik Indonesia.",Kapan masa berlaku peraturan ini dimulai menurut Pasal 20?,Kapan masa berlaku peraturan ini menurut Pasal 20?,Apa perubahan regulasi pada Pasal 20?,Apa saja perubahan regulasi dalam Pasal 20?,https://www.ojk.go.id/id/regulasi/Documents/Pages/Penilaian-Tingkat-Kesehatan-Bank-Umum-Syariah-dan-Unit-Usaha-Syariah/POJK_1404381860.pdf,https://www.ojk.go.id/id/regulasi/Documents/Pages/POJK-tentang-Penilaian-Tingkat-Kesehatan-Bank-Umum-Syariah-dan-Unit-Usaha-Syariah/pojk%208-2014.pdf


In [50]:
## 6) Union-Find Clustering on dup_pairs
parent = list(range(len(df)))

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

for a, b in dup_pairs:
    union(a, b)

## 7) Build Clusters & Select Representatives
clusters = {}
for idx in range(len(df)):
    root = find(idx)
    clusters.setdefault(root, []).append(idx)

to_keep = []
for members in clusters.values():
    if len(members) == 1:
        to_keep.append(members[0])
    else:
        # pilih idx dengan context paling panjang
        best = max(members, key=lambda i: len(df.loc[i, 'context']))
        to_keep.append(best)

## 8) Filter & Save Cleaned Data
df_clean = df.loc[sorted(to_keep)].reset_index(drop=True)
print(f"\nAfter filtering clusters: {len(df_clean)} pairs remain (from {len(df)})")
output_clean = "cqa_datasets_clean.jsonl"
df_clean.to_json(output_clean, orient="records", lines=True, force_ascii=False)
print(f"→ Saved cleaned data to {output_clean}\n")


After filtering clusters: 1694 pairs remain (from 1696)
→ Saved cleaned data to cqa_datasets_clean.jsonl



## **EDA Preprocessing**

In [51]:
df_clean = pd.read_json("cqa_datasets_clean.jsonl", lines=True)
print(f"Total QA pairs: {len(df_clean)}")

Total QA pairs: 1694


In [52]:
df_clean['category_answerable'].value_counts()

category_answerable
Ketentuan Umum yang Ditetapkan    156
Kewajiban yang Ditetapkan         152
Cakupan Dokumen                   151
Mekanisme Proses                  149
Subjek yang Diatur                149
Kategori Penilaian                148
Objek yang Diatur                 147
Waktu Berlaku                     138
Perubahan Regulasi                128
Pengertian                        127
Mekanisme Pelaporan                85
Dasar Hukum                        57
Sanksi Hukum                       54
Tujuan                             53
Name: count, dtype: int64

In [53]:
df_clean['category_unanswerable'].value_counts()

category_unanswerable
Tujuan                            159
Mekanisme Pelaporan               151
Sanksi Hukum                      148
Dasar Hukum                       147
Pengertian                        147
Perubahan Regulasi                138
Kategori Penilaian                134
Waktu Berlaku                     131
Mekanisme Proses                  123
Ketentuan Umum yang Ditetapkan    105
Kewajiban yang Ditetapkan          93
Subjek yang Diatur                 88
Cakupan Dokumen                    68
Objek yang Diatur                  62
Name: count, dtype: int64

In [54]:
# kategori = [
#     "Masa Berlaku",
#     "Larangan",
#     "Pengawasan",
#     "Ketentuan Peralihan",
#     "Tugas dan Tanggung Jawab",
#     "Kriteria Penilaian/Status",
#     "Tugas, Wewenang, Hak, dan Tanggung Jawab",
#     "Hak",
#     "Biaya"
# ]

# df_clean[df_clean['category'].isin(kategori)]


## **Redefine Categories**

In [55]:
# Daftar kategori yang kita definisikan di prompt
prompt_categories = [
    "Pengertian", "Tujuan", "Subjek yang Diatur", "Objek yang Diatur",
    "Kewajiban yang Ditetapkan", "Ketentuan Umum yang Ditetapkan",
    "Mekanisme Pelaporan", "Mekanisme Proses", "Kategori Penilaian",
    "Sanksi Hukum", "Perubahan Regulasi", "Waktu Berlaku",
    "Cakupan Dokumen", "Dasar Hukum"
]

# Ganti semua kategori lain menjadi 'Others'
df_clean['category_answerable'] = df_clean['category_answerable'].where(
    df_clean['category_answerable'].isin(prompt_categories),
    other="Others"
)

df_clean['category_unanswerable'] = df_clean['category_unanswerable'].where(
    df_clean['category_unanswerable'].isin(prompt_categories),
    other="Others"
)

In [56]:
# Tampilkan distribusi baru
print("Distribusi kategori setelah redefinisi:")
print(df_clean['category_answerable'].value_counts())

Distribusi kategori setelah redefinisi:
category_answerable
Ketentuan Umum yang Ditetapkan    156
Kewajiban yang Ditetapkan         152
Cakupan Dokumen                   151
Mekanisme Proses                  149
Subjek yang Diatur                149
Kategori Penilaian                148
Objek yang Diatur                 147
Waktu Berlaku                     138
Perubahan Regulasi                128
Pengertian                        127
Mekanisme Pelaporan                85
Dasar Hukum                        57
Sanksi Hukum                       54
Tujuan                             53
Name: count, dtype: int64


In [57]:
# Tampilkan distribusi baru
print("Distribusi kategori setelah redefinisi:")
print(df_clean['category_unanswerable'].value_counts())

Distribusi kategori setelah redefinisi:
category_unanswerable
Tujuan                            159
Mekanisme Pelaporan               151
Sanksi Hukum                      148
Dasar Hukum                       147
Pengertian                        147
Perubahan Regulasi                138
Kategori Penilaian                134
Waktu Berlaku                     131
Mekanisme Proses                  123
Ketentuan Umum yang Ditetapkan    105
Kewajiban yang Ditetapkan          93
Subjek yang Diatur                 88
Cakupan Dokumen                    68
Objek yang Diatur                  62
Name: count, dtype: int64


In [58]:
# (Opsional) Simpan hasilnya
df_clean.to_json("cqa_datasets_clean_recat.jsonl", orient="records", lines=True, force_ascii=False)
print("→ Saved recategorized data to cqa_datasets_clean_recat.jsonl")

→ Saved recategorized data to cqa_datasets_clean_recat.jsonl


## **Split Data into 3 Parts (SFT, RL, Test)**

- SFT: 60% of the data
- RL: 30% of the data
- Testing: 10% of the data

In [59]:
# 0) Buat kolom gabungan
df_clean['stratify_key'] = (
    df_clean['category_answerable'].astype(str)
    + "_"
    + df_clean['category_unanswerable'].astype(str)
)

# 0.1) Group rare keys (<2) into 'Other_Other'
key_counts = df_clean['stratify_key'].value_counts()
df_clean['stratify_key'] = df_clean['stratify_key'].apply(
    lambda k: k if key_counts[k] >= 2 else 'Other_Other'
)

# 1) Split train_val vs test (10%)
train_val, test = train_test_split(
    df_clean,
    test_size=0.10,
    random_state=42,
    shuffle=True,
    stratify=df_clean['stratify_key']
)

# 2) Split train_val → SFT (20%) & RL (70%)
sft, rl = train_test_split(
    train_val,
    test_size=(0.70 / 0.90), # juga dilakukan SFT (45%) & RL (45%)
    random_state=42,
    shuffle=True,
    stratify=train_val['stratify_key']
)

# 3) Laporkan
print(f"SFT:  {len(sft)} ({len(sft)/len(df_clean):.1%})")
print(f"RL:   {len(rl)} ({len(rl)/len(df_clean):.1%})")
print(f"Test: {len(test)} ({len(test)/len(df_clean):.1%})")

# 4) Drop kolom bantu dan simpan
for df_split, name in [(sft, "sft"), (rl, "rl"), (test, "test")]:
    df_split = df_split.drop(columns='stratify_key')
    df_split.to_json(f"cqa_{name}.jsonl", orient="records", lines=True, force_ascii=False)
    print(f"→ Saved {name} ({len(df_split)})")

SFT:  338 (20.0%)
RL:   1186 (70.0%)
Test: 170 (10.0%)
→ Saved sft (338)
→ Saved rl (1186)
→ Saved test (170)


In [60]:
# 1) Load kembali split kalau belum ada di memori
sft  = pd.read_json("cqa_sft.jsonl",  lines=True)
rl   = pd.read_json("cqa_rl.jsonl",   lines=True)
test = pd.read_json("cqa_test.jsonl", lines=True)

# 2) Fungsi helper untuk print distribusi
def print_dist(name, df):
    print(f"\n=== {name} split ===")
    print("-> category_answerable:")
    print(df['category_answerable'].value_counts(dropna=False).to_frame('count'))
    print("\n-> category_unanswerable:")
    print(df['category_unanswerable'].value_counts(dropna=False).to_frame('count'))

# 3) Cetak untuk semua split
print_dist("SFT",  sft)
print_dist("RL",   rl)
print_dist("Test", test)


=== SFT split ===
-> category_answerable:
                                count
category_answerable                  
Ketentuan Umum yang Ditetapkan     33
Subjek yang Diatur                 32
Mekanisme Proses                   31
Cakupan Dokumen                    30
Kategori Penilaian                 30
Kewajiban yang Ditetapkan          29
Objek yang Diatur                  29
Waktu Berlaku                      29
Pengertian                         25
Perubahan Regulasi                 24
Mekanisme Pelaporan                15
Dasar Hukum                        11
Tujuan                             10
Sanksi Hukum                       10

-> category_unanswerable:
                                count
category_unanswerable                
Tujuan                             31
Mekanisme Pelaporan                30
Pengertian                         30
Sanksi Hukum                       29
Dasar Hukum                        29
Kategori Penilaian                 29
Waktu Berlaku     

## **Separate Asnwerable and Unanswerable**
Separate Asnwerable and Unanswerable [context, question_answerable, answer_answerable, question_unanswerable, answer_unanswerable] into [{context, question_answerable, answer_answerable}, {context, question_unanswerable, answer_unanswerable}, ...] for SFT, RL, and Test.

In [61]:
# 1) Load the saved splits
sft = pd.read_json("cqa_sft.jsonl", lines=True)
rl  = pd.read_json("cqa_rl.jsonl",  lines=True)
test= pd.read_json("cqa_test.jsonl",lines=True)

# 2) Define helper to expand each row into two entries
def separate_entries(df_split):
    records = []
    for _, row in df_split.iterrows():
        # answerable entry
        records.append({
            "context": row["context"],
            "question": row["question_answerable"],
            "answer":   row["answer_answerable"],
            "category": row["category_answerable"]
        })
        # unanswerable entry
        records.append({
            "context": row["context"],
            "question": row["question_unanswerable"],
            "answer":   row["answer_unanswerable"],
            "category": row["category_unanswerable"]
        })
    return records

# 3) Expand and save each split
for split_name, split_df in [("sft", sft), ("rl", rl), ("test", test)]:
    expanded = separate_entries(split_df)
    out_file = f"cqa_{split_name}_expanded.jsonl"
    with open(out_file, "w", encoding="utf-8") as f:
        for rec in expanded:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")
    print(f"→ Saved {len(expanded)} entries for {split_name.upper()} to {out_file}")

→ Saved 676 entries for SFT to cqa_sft_expanded.jsonl
→ Saved 2372 entries for RL to cqa_rl_expanded.jsonl
→ Saved 340 entries for TEST to cqa_test_expanded.jsonl


In [62]:
# Load the expanded datasets
sft_expanded  = pd.read_json("cqa_sft_expanded.jsonl",  lines=True)
rl_expanded   = pd.read_json("cqa_rl_expanded.jsonl",   lines=True)
test_expanded = pd.read_json("cqa_test_expanded.jsonl", lines=True)

# Print the number of entries in each
print(f"SFT expanded entries:  {len(sft_expanded)}")
print(f"RL  expanded entries:  {len(rl_expanded)}")
print(f"Test expanded entries: {len(test_expanded)}")

SFT expanded entries:  676
RL  expanded entries:  2372
Test expanded entries: 340


In [63]:
# Fungsi untuk menghitung jumlah answerable vs unanswerable berdasarkan urutan baris
def count_answerability_by_order(df, name):
    total = len(df)
    answerable_count = total // 2
    unanswerable_count = total // 2  # karena selalu 2 baris per QA-pair
    
    print(f"\n=== {name.upper()} ===")
    print(f"Total entries: {total}")
    print(f"→ Answerable (genap index):     {answerable_count} ({answerable_count / total:.1%})")
    print(f"→ Unanswerable (ganjil index):  {unanswerable_count} ({unanswerable_count / total:.1%})")

    print("\nDistribusi kategori:")
    print(df['category'].value_counts(dropna=False).to_frame('count'))

# Tampilkan untuk masing-masing split
count_answerability_by_order(sft_expanded, "sft")
count_answerability_by_order(rl_expanded, "rl")
count_answerability_by_order(test_expanded, "test")


=== SFT ===
Total entries: 676
→ Answerable (genap index):     338 (50.0%)
→ Unanswerable (ganjil index):  338 (50.0%)

Distribusi kategori:
                                count
category                             
Kategori Penilaian                 59
Waktu Berlaku                      57
Mekanisme Proses                   56
Pengertian                         55
Ketentuan Umum yang Ditetapkan     53
Perubahan Regulasi                 51
Subjek yang Diatur                 49
Kewajiban yang Ditetapkan          49
Mekanisme Pelaporan                45
Cakupan Dokumen                    43
Tujuan                             41
Dasar Hukum                        40
Objek yang Diatur                  39
Sanksi Hukum                       39

=== RL ===
Total entries: 2372
→ Answerable (genap index):     1186 (50.0%)
→ Unanswerable (ganjil index):  1186 (50.0%)

Distribusi kategori:
                                count
category                             
Kategori Penilaian            

### **Prompt-Completion Creation**

from the data SFT, RL, and Test, create the prompt-completion pairs for the model.

In [64]:

def make_prompt_completion(df_in, out_file):
    """
    Given a DataFrame with columns ['context', 'question', 'answer'], 
    create a JSONL file with prompt/completion pairs including a system instruction:
      - system instruction: You are an Indonesian financial regulation expert...
      - prompt: "<SYSTEM_INSTRUCTION>\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"
      - completion: " {answer}"
    """
    system_instr = (
        "Anda adalah pakar regulasi keuangan Indonesia. "
        "Jawablah berdasarkan konteks yang disediakan; "
        "jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”"
    )
    with open(out_file, 'w', encoding='utf-8') as fout:
        for _, row in df_in.iterrows():
            prompt = (
                f"{system_instr}\n\n"
                f"Context:\n{row['context']}\n\n"
                f"Question: {row['question']}\n"
                f"Answer:"
            )
            completion = f" {row['answer']}"
            entry = {
                "prompt": prompt,
                "completion": completion
            }
            fout.write(json.dumps(entry, ensure_ascii=False) + "\n")

# Load expanded datasets and generate prompt-completion files
for split in ['sft', 'rl', 'test']:
    df_split = pd.read_json(f"cqa_{split}_expanded.jsonl", lines=True)
    out_file = f"cqa_{split}_prompt_completion.jsonl"
    make_prompt_completion(df_split, out_file)
    print(f"→ Created {len(df_split)} prompt-completion pairs in {out_file}")


→ Created 676 prompt-completion pairs in cqa_sft_prompt_completion.jsonl
→ Created 2372 prompt-completion pairs in cqa_rl_prompt_completion.jsonl
→ Created 340 prompt-completion pairs in cqa_test_prompt_completion.jsonl


In [65]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

# 1) Load the test prompt-completion dataset
df_test = pd.read_json("cqa_test_prompt_completion.jsonl", lines=True)

# 2) Ambil 5 sampel acak
sampled = df_test.sample(5).reset_index(drop=True)

# 3) Tampilkan
print(sampled['prompt'][0])


Anda adalah pakar regulasi keuangan Indonesia. Jawablah berdasarkan konteks yang disediakan; jika tidak terdapat pada konteks, jawab “Saya tidak tahu terkait {question}.”

Context:
Mengingat : 1. Undang-Undang Nomor 7 Tahun 1992 tentang Perbankan (Lembaran Negara Republik Indonesia Tahun 1992  Nomor 31, Tambahan Lembaran Negara Republik Indonesia Nomor 3472) sebagaimana telah diubah dengan Undang-Undang Nomor 10 Tahun 1998 tentang Perubahan atas Undang-Undang Nomor 7 Tahun 1992 tentang Perbankan (Lembaran Negara Republik Indonesia Tahun 1998 Nomor 182, Tambahan Lembaran Negara Republik Indonesia Nomor 3970); 2. Undang-Undang Nomor 8 Tahun 1995 tentang Pasar Modal (Lembaran Negara Republik Indonesia Tahun 1995  Nomor 64, Tambahan Lembaran Negara Republik Indonesia Nomor 3608); 3. Undang-Undang Nomor 21 Tahun 2008 tentang Perbankan Syariah (Lembaran Negara Republik Indonesia Tahun 2008 Nomor 94, Tambahan Lembaran Negara Republik Indonesia Nomor 4867); 4. Undang-Undang Nomor 21 Tahun 2011

In [66]:
# Combine dataset cqa_sft_prompt_completion.jsonl and cqa_rl_prompt_completion.jsonl into a single file with a new name cqa_full_training_prompt_completion.jsonl
import pandas as pd

# Load the two datasets
sft_data = pd.read_json("cqa_sft_prompt_completion.jsonl", lines=True)
rl_data  = pd.read_json("cqa_rl_prompt_completion.jsonl", lines=True)

# Combine them
full_train_data = pd.concat([sft_data, rl_data], ignore_index=True)

# Save to a new file
output_file = "cqa_full_training_prompt_completion.jsonl"
full_train_data.to_json(output_file, orient="records", lines=True, force_ascii=False)

print(f"→ Combined dataset saved to {output_file} with {len(full_train_data)} prompt-completion pairs")

→ Combined dataset saved to cqa_full_training_prompt_completion.jsonl with 3048 prompt-completion pairs
